### Hybrid Search Langchain

In [1]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [2]:
api_key = "056702d839c7-e5b6e6-44ea-a25f-b337d0570982"

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

Creating the INDEX

In [4]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name ="hybrid-search-langchain-pinecone"

## initialize the pinecone client
pc=Pinecone(api_key=api_key)


## create the index
if index_name not in pc.list_indexes():
    pc.create_index(name=index_name,
                    dimension=384, #dimension of dense vector
                    metric="dotproduct", ## sparse values supported only for dotproduct
                    spec=ServerlessSpec(cloud='aws', region='us-east-1'),
                    )
    

In [5]:
index=pc.Index(index_name)
index

Dense vector embeddings using Hugging Face Embeddings

In [7]:
## vector embedding and sparse matrix

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

d:\miniconda\envs\hybrid\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BASIM ASLAM\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

Sparse vector embeddings using BM25

In [8]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [9]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited London",
    "In 2021, I visited New York",
    "In 2020, I visited Tokyo",
]

# best match 25 bulid on top of tf-idf
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

## load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 4/4 [00:00<00:00, 11.48it/s]


Combining Both

In [12]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [13]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001CE983A5A50>, index=<pinecone.data.index.Index object at 0x000001CEEA668190>)

In [14]:
retriever.add_texts(
    [
        "In 2023, I visited Paris",
        "In 2022, I visited London",
        "In 2021, I visited New York",
        "In 2020, I visited Tokyo",
    ]
)

100%|██████████| 1/1 [00:04<00:00,  4.66s/it]


In [15]:
retriever.invoke("what city did I visit in 2022?")

[Document(page_content='In 2022, I visited London'),
 Document(page_content='In 2021, I visited New York'),
 Document(page_content='In 2023, I visited Paris'),
 Document(page_content='In 2020, I visited Tokyo')]